# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install tensorflow-hub
!pip install opencv-python
!pip install mediapipe
!pip install sklearn
!pip install matplotlib

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [39 lines of output]
  Traceback (most recent call last):
    File "C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
      parsed = _parse_requirement(requirement_string)
    File "C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
    File "C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
    File "C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 126, in _parse_requirement_details
      

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import os

In [3]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print(gpus)

[]


# 2. Keypoints

## 2.1 Using MediaPipe

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR TO RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR CONVERSION RGB TO BGR
    return image, results

In [6]:
def draw_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
    #                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
    #                         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    # mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
    #                         mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
    #                         mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
    #                          ) 
    # Draw right hand connections
    # mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
    #                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
    #                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
    #                          ) 
    

In [7]:
cap = cv2.VideoCapture(0)

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
    
        # Read feed
        ret, frame = cap.read()
    
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
    
        draw_landmarks(image, results)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
    
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

### Note : 
MediaPipe is only able to detect a single person's pose and body, which is not what we are trying to do!

## 2.2 Using MoveNet

### 2.2.1 Load Model

### 2.2.2 Draw Keypoints

### 2.2.3 Draw Edges

### 2.2.4 Make Detections

# 3. Extract keypoints Values

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros()
    # lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    # rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    # print(pose.shape, face.shape, lh.shape, rh.shape)
    # print(pose, face, lh, rh, sep='\n')
    # return np.concatenate([pose, face, lh, rh])
    return pose

# 4. Setup Folders for Collection

In [9]:
# Path for exported data, numpy arrays
VIDEO_PATH = os.path.join('Data')
DATA_PATH = os.path.join('Pose Data')

try:
    os.mkdir(DATA_PATH)
except:
    pass

# Actions that we try to detect
actions = np.array(["Punch", "Crawl", "Run", "Walk"])

for action in actions:
    try:
        os.mkdir(os.path.join(DATA_PATH, action))
    except:
        pass

# Videos are going to be 30 frames in length
sequence_length = 30

# 5. Collect Keypoint Values for Training and Testing

In [10]:
def save_data(results, name, confidence_threshold):
        keypoints = extract_keypoints(results)
        # keypoints = extract_data(results, confidence_threshold)
        # print("Filtering ...")

        # print(keypoints)
    
        npy_path = os.path.join(DATA_PATH, 'Jump', name)
        # print(npy_path)
        np.save(npy_path, keypoints)

In [287]:
cap = cv2.VideoCapture(os.path.join('Data', 'Jump', 'Jump.mp4'))

# seq = 1

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
            # Read feed
            ret, frame = cap.read()
    
            if frame is None:
                break
                
            # # Resize image
            # img = frame.copy()
            # img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 160, 256)
            # input_img = tf.cast(img, dtype=tf.int32)
        
            # # Make detections
            # results = movenet(input_img)
    
            # # Render Keypoints
            # loop_through_people(frame, results, EDGES, 0.3)

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_landmarks(image, results)
        
            # Show to screen
            cv2.imshow('OpenCV Feed', image)
        
            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    
            if cv2.waitKey(10) & 0xFF == 27:
                break
    
            # save_data(results, str(seq), 0.3)
            
            seq+=1
    cap.release()
    cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'Pose Data\\Jump\\0.npy'

In [288]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'Punch': 0, 'Crawl': 1, 'Run': 2, 'Walk': 3}

In [13]:
from tqdm import tqdm

sequences, labels = [], []
window = []

for action in actions:
    seq = 0
    count = 0
    for sequence in tqdm(range(1, 4800)):
        # print(sequence)
        res = np.load(os.path.join(DATA_PATH, action, f"{sequence}.npy"))
        # res = res.reshape(306)
        window.append(res)
        seq+=1
        count+=1
        if seq >= 30:
            sequences.append(np.array(window))
            labels.append(label_map[action])
            window = []
            seq = 0
            if count>=4580:
                print(action)
                break
        

 96%|████████████████████████████████████████████████████████████████████████▋   | 4589/4799 [00:02<00:00, 1557.77it/s]


Punch


 96%|████████████████████████████████████████████████████████████████████████▋   | 4589/4799 [00:02<00:00, 1713.35it/s]


Crawl


 96%|████████████████████████████████████████████████████████████████████████▋   | 4589/4799 [00:02<00:00, 1581.34it/s]


Run


 96%|████████████████████████████████████████████████████████████████████████▋   | 4589/4799 [00:03<00:00, 1446.05it/s]

Walk


In [14]:
X = np.array(sequences)
X.shape

(612, 30, 132)

In [15]:
y = to_categorical(labels).astype(int)
y.shape

(612, 4)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 7. Build and Train LSTM Neural Network

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [40]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [41]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 132)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [42]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [43]:
tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='min',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)

In [44]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
19/19 [==============================] - 4s 34ms/step - loss: 1.1035 - categorical_accuracy: 0.4957
Epoch 2/500
19/19 [==============================] - 1s 33ms/step - loss: 0.9213 - categorical_accuracy: 0.8554
Epoch 3/500
19/19 [==============================] - 1s 34ms/step - loss: 0.5943 - categorical_accuracy: 0.7625
Epoch 4/500
19/19 [==============================] - 1s 34ms/step - loss: 0.4173 - categorical_accuracy: 0.8365
Epoch 5/500
19/19 [==============================] - 1s 34ms/step - loss: 0.2293 - categorical_accuracy: 0.9535
Epoch 6/500
19/19 [==============================] - 1s 34ms/step - loss: 0.1044 - categorical_accuracy: 0.9690
Epoch 7/500
19/19 [==============================] - 1s 34ms/step - loss: 0.1763 - categorical_accuracy: 0.9398
Epoch 8/500
19/19 [==============================] - 1s 34ms/step - loss: 0.1470 - categorical_accuracy: 0.9621
Epoch 9/500
19/19 [==============================] - 1s 34ms/step - loss: 0.1630 - categorical_accuracy:

KeyboardInterrupt: 

# 8. Make Predictions

In [45]:
res = model.predict(X_test)

1/1 [==============================] - 0s 356ms/step


In [46]:
actions[np.argmax(res[6])]

'Run'

In [47]:
actions[np.argmax(y_test[6])]

'Run'

# 9. Save Weights

In [55]:
model.save('Anomaly_new.h5')

C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 10. Evaluation using Confusion Matrix and Accuracy

In [48]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [49]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 31ms/step


In [50]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [51]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[23,  0],
        [ 0,  8]],

       [[28,  0],
        [ 0,  3]],

       [[20,  1],
        [ 0, 10]],

       [[21,  0],
        [ 1,  9]]], dtype=int64)

In [52]:
accuracy_score(ytrue, yhat)

0.967741935483871

# 11. Test in Real Time

In [53]:
from scipy import stats

colors = [(16,117,245), (117,245,16)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,40+num*30), (int(prob*100), 70+num*30), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 60+num*30), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (255,255,255), 1, cv2.LINE_AA)
        
    return output_frame

In [54]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

# video = cv2.VideoWriter('Video_result.avi',  
#                          cv2.VideoWriter_fourcc(*'MJPG'), 
#                          10, (640, 480) )

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
    
        # Read feed
        ret, frame = cap.read()
    
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
    
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        # keypoints = keypoints.reshape(306)
        # print(keypoints.shape)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
    
            if len(sentence) > 5: 
                sentence = sentence[-5:]
    
            # Viz probabilities
            image = prob_viz([max(res[:-1]), res[-1]], ['Anomaly', 'Normal'], image, colors)
            
        cv2.rectangle(image, (0,0), (640, 30), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 20), 
                       cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # video.write(image)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
    
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step
Crawl
1/1 [==============================] - 0s 23ms/step
Crawl
1/1 [==============================] - 0s 23ms/step
Crawl
1/1 [==============================] - 0s 23ms/step
Crawl
1/1 [==============================] - 0s 22ms/step
Crawl
1/1 [==============================] - 0s 22ms/step
Crawl
1/1 [==============================] - 0s 22ms/step
Walk
1/1 [==============================] - 0s 23ms/step
Walk
1/1 [==============================] - 0s 22ms/step
Walk
1/1 [==============================] - 0s 23ms/step
Walk
1/1 [==============================] - 0s 21ms/step
Walk
1/1 [==============================] - 0s 22ms/step
Walk
1/1 [==============================] - 0s 23ms/step
Walk
1/1 [==============================] - 0s 22ms/step
Walk
1/1 [==============================] - 0s 21ms/step
Crawl
1/1 [==============================] - 0s 22ms/step
Crawl
1/1 [==============================] - 0s 21ms/step
Crawl
1/1 [=================

In [186]:
res

array([1.8742521e-12, 3.0496388e-14, 1.0000000e+00], dtype=float32)

In [316]:
cap.release()
cv2.destroyAllWindows()
video.release() 

In [261]:
frame.shape

(480, 640, 3)